In [1]:
import csv
import collections
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path
from sklearn.model_selection import train_test_split
import time
import pygraphviz
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample

In [2]:
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=False, **kwargs):
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        start = time.time()
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...{i / len(self.labels) * 100}%")
            print(f"{(time.time() - start) / 60} minutes")
            predictor.fit(train_data=train_data[train_data[label] > float('-inf')]
                          , tuning_data = tuning_data
                          ,presets = 'best_quality'
                          ,num_bag_folds = 5,num_bag_sets = 2
                          ,hyperparameter_tune_kwargs =  {
                              'num_trials': 128,
                              'searcher': 'random',
                              'scheduler' : 'local'
                          }
                          , **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            
            eval_dict[label] = predictor.evaluate(data[data[label] > float('-inf')], **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for i,label in enumerate(self.labels):
            print(f"Predicting with TabularPredictor for label: {label} ...{i / len(self.labels) * 100}%")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [3]:
#1 Load my ccl's ssGSEA signature
myCCLSignature = []
for name in ['sample.c2.cp.biocarta.gct',
             'sample.c2.cp.kegg.gct',
             'sample.c2.cp.pid.gct',
             'sample.c2.cp.reactome.gct',
             'sample.c2.cp.wiki.gct',
             'sample.c6.gct',
             'sample.hallmark.gct']:    
    with open(name, mode ='r')as file:
        csvFile = csv.reader(file)
        CCLSignature = list(csvFile)[2:]
        print(len(CCLSignature))
    for i, row in enumerate(CCLSignature):
        temp = CCLSignature[i][0].split('\t')
        if i > 0:
            CCLSignature[i] = [temp[0]] + [float(d) for d in temp[2:]]
        else:
            CCLSignature[i] = [temp[0]] + temp[2:]
    if not myCCLSignature:
        myCCLSignature += CCLSignature
    else:
        myCCLSignature += CCLSignature[1:]

293
187
197
1616
665
280
53


In [4]:
#2 Load CCLE ssGSEA signature
CCLECCLSignature = []
for name in ['ccle.c2.cp.biocarta.gct',
             'ccle.c2.cp.kegg.gct',
             'ccle.c2.cp.pid.gct',
             'ccle.c2.cp.reactome.gct',
             'ccle.c2.cp.wiki.gct',
             'ccle.c6.gct',
             'ccle.hallmark.gct']:
    with open(name, mode ='r')as file:
        csvFile = csv.reader(file)
        CCLSignature = list(csvFile)[2:]
        print(len(CCLSignature))
    for i, row in enumerate(CCLSignature):
        temp = CCLSignature[i][0].split('\t')
        CCLSignature[i] = [temp[0]] + temp[2:]
    if not CCLECCLSignature:
        CCLECCLSignature += CCLSignature
    else:
        CCLECCLSignature += CCLSignature[1:]

293
187
197
1616
665
280
53


In [5]:
#3 Load CTRP cclName to AUC map
cclToAUCdict = collections.defaultdict(list)
with open('CTRP_CCL_AUC.gct', mode ='r') as file:
    csvFile = csv.reader(file)
    CTRPCCLAUC = list(csvFile)
    CTRPCCLAUC = [''.join(sub).split('\t') for sub in CTRPCCLAUC]
    cclNames = CTRPCCLAUC[3][4:]

for i,cclName in enumerate(cclNames):
    cclToAUCdict[cclName] = [float( '-inf' if sub[4+i] == 'NaN' else sub[4+i]) for sub in CTRPCCLAUC[7:]]

In [6]:
#4 Load ccleID to ctrpName map
CCLEidToCTRPNameDict = collections.defaultdict(str)
CCLEidToDiseaseName = collections.defaultdict(str)
with open('sample_info.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    mapInfos = list(csvFile)
    for mapInfo in mapInfos[1:]:
        CCLEidToCTRPNameDict[mapInfo[0]] = mapInfo[2]  
        CCLEidToDiseaseName[mapInfo[0]] = mapInfo[12]

In [7]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [8]:
#5 Prepare train set
trainData = pd.DataFrame(data = CCLECCLSignature).transpose()
new_header = trainData.iloc[0] 
trainData = trainData[1:] 
trainData.columns = new_header 

In [9]:
#6 filter valid ID
validSet = set()
for name in trainData['Name']:
    if CCLEidToCTRPNameDict[name] in cclToAUCdict:
        validSet.add(name)
trainData = trainData.loc[trainData['Name'].isin(validSet)]
trainData = trainData.reset_index(drop = True)
trainData = trainData.set_index(['Name'])
trainData = trainData.apply(pd.to_numeric)
trainData = normalize(trainData)
trainData.head()

,BIOCARTA_GRANULOCYTES_PATHWAY,BIOCARTA_LYM_PATHWAY,BIOCARTA_BLYMPHOCYTE_PATHWAY,BIOCARTA_CARM_ER_PATHWAY,BIOCARTA_LAIR_PATHWAY,BIOCARTA_VDR_PATHWAY,BIOCARTA_MTA3_PATHWAY,BIOCARTA_GABA_PATHWAY,BIOCARTA_EGFR_SMRTE_PATHWAY,BIOCARTA_MONOCYTE_PATHWAY,...,HALLMARK_COAGULATION,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS
Name,,,,,,,,,,,,,,,,,,,,,
ACH-001113,0.201899,0.214778,0.107506,0.436254,0.251323,0.543866,0.402230,0.446239,0.692287,0.299681,...,0.369956,0.589361,0.231132,0.508302,0.178205,0.653705,0.345327,0.352541,0.390359,0.232997
ACH-000242,0.210679,0.195089,0.137566,0.598515,0.192437,0.888515,0.381405,0.108177,0.679700,0.285389,...,0.448744,0.651476,0.461799,0.710828,0.337337,0.380484,0.503869,0.462545,0.288830,0.128070
ACH-000327,0.226109,0.228415,0.131664,0.807736,0.214956,0.105343,0.366447,0.534534,0.508604,0.279435,...,0.377387,0.627130,0.639671,0.951057,0.237352,0.491974,0.441105,0.435839,0.353783,0.339955
ACH-000461,0.230496,0.359472,0.087643,0.529038,0.244749,0.476536,0.201961,0.313338,0.724363,0.298408,...,0.475189,0.725638,0.126653,0.531472,0.348210,0.451362,0.611400,0.517533,0.191123,0.196242
ACH-000792,0.394568,0.663835,0.308974,0.400182,0.578346,0.287388,0.289313,0.165095,0.557640,0.630891,...,0.624730,0.829904,0.180853,0.480622,0.450919,0.481082,0.591145,0.458782,0.138390,0.197292


In [10]:
labelsData = pd.DataFrame(columns = [sub[1] for sub in CTRPCCLAUC[7:]])
for name in trainData.index:
    labelsData.loc[len(labelsData.index)] = cclToAUCdict[CCLEidToCTRPNameDict[name]]
labelsData = labelsData.set_index(trainData.index)

In [11]:
trainDataSet = pd.concat([trainData,labelsData],axis = 1)
trainDataSet.head()

,BIOCARTA_GRANULOCYTES_PATHWAY,BIOCARTA_LYM_PATHWAY,BIOCARTA_BLYMPHOCYTE_PATHWAY,BIOCARTA_CARM_ER_PATHWAY,BIOCARTA_LAIR_PATHWAY,BIOCARTA_VDR_PATHWAY,BIOCARTA_MTA3_PATHWAY,BIOCARTA_GABA_PATHWAY,BIOCARTA_EGFR_SMRTE_PATHWAY,BIOCARTA_MONOCYTE_PATHWAY,...,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS,dasatinib
Name,,,,,,,,,,,,,,,,,,,,,
ACH-001113,0.201899,0.214778,0.107506,0.436254,0.251323,0.543866,0.402230,0.446239,0.692287,0.299681,...,0.589361,0.231132,0.508302,0.178205,0.653705,0.345327,0.352541,0.390359,0.232997,12.64
ACH-000242,0.210679,0.195089,0.137566,0.598515,0.192437,0.888515,0.381405,0.108177,0.679700,0.285389,...,0.651476,0.461799,0.710828,0.337337,0.380484,0.503869,0.462545,0.288830,0.128070,10.44
ACH-000327,0.226109,0.228415,0.131664,0.807736,0.214956,0.105343,0.366447,0.534534,0.508604,0.279435,...,0.627130,0.639671,0.951057,0.237352,0.491974,0.441105,0.435839,0.353783,0.339955,14.59
ACH-000461,0.230496,0.359472,0.087643,0.529038,0.244749,0.476536,0.201961,0.313338,0.724363,0.298408,...,0.725638,0.126653,0.531472,0.348210,0.451362,0.611400,0.517533,0.191123,0.196242,9.68
ACH-000792,0.394568,0.663835,0.308974,0.400182,0.578346,0.287388,0.289313,0.165095,0.557640,0.630891,...,0.829904,0.180853,0.480622,0.450919,0.481082,0.591145,0.458782,0.138390,0.197292,12.28


In [12]:
#7 prediction
predictData = pd.DataFrame(data = myCCLSignature).transpose()
new_header = predictData.iloc[0] 
predictData = predictData[1:] 
predictData.columns = new_header 
predictData = predictData.apply(pd.to_numeric, errors='ignore')
predictData = predictData.set_index(['Name'])
predictData = normalize(predictData)
predictData.head()

,BIOCARTA_GRANULOCYTES_PATHWAY,BIOCARTA_LYM_PATHWAY,BIOCARTA_BLYMPHOCYTE_PATHWAY,BIOCARTA_CARM_ER_PATHWAY,BIOCARTA_LAIR_PATHWAY,BIOCARTA_VDR_PATHWAY,BIOCARTA_MTA3_PATHWAY,BIOCARTA_GABA_PATHWAY,BIOCARTA_EGFR_SMRTE_PATHWAY,BIOCARTA_MONOCYTE_PATHWAY,...,HALLMARK_COAGULATION,HALLMARK_IL2_STAT5_SIGNALING,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_PEROXISOME,HALLMARK_ALLOGRAFT_REJECTION,HALLMARK_SPERMATOGENESIS,HALLMARK_KRAS_SIGNALING,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_DN,HALLMARK_PANCREAS_BETA_CELLS
Name,,,,,,,,,,,,,,,,,,,,,
THP1_d3_B.TPM,0.991211,0.985279,0.984154,0.876359,0.993570,0.595547,0.475054,0.492397,0.594030,0.986427,...,0.675939,0.809839,0.680037,0.774811,0.928606,0.793392,1.000000,0.880986,0.337452,0.670398
2D_1.TPM,0.160241,0.161473,0.059402,0.379321,0.045364,1.000000,0.089837,0.554072,0.363435,0.187854,...,0.025288,0.065203,0.221749,0.194506,0.019127,0.555587,0.000000,0.037313,0.343351,0.248108
TRId7CWa.quant.TPM,0.438166,0.443539,0.540397,0.000000,0.506852,0.304020,0.583516,0.429940,1.000000,0.700186,...,0.927243,0.736486,0.484722,0.130770,0.436573,0.053547,0.354081,0.771492,0.968649,0.720934
H7.TPM,0.553921,0.562553,0.428064,1.000000,0.724494,0.403809,0.852908,0.493875,0.511236,0.531963,...,0.822794,0.816438,0.113695,0.248351,0.476935,0.189215,0.544488,0.707475,0.643843,0.258149
THP1_d3_A.TPM,0.992471,0.992554,0.973644,0.795964,0.999802,0.548292,0.615737,0.447267,0.474747,0.989311,...,0.691364,0.826434,0.710767,0.877729,0.928190,0.791363,0.947641,0.933168,0.476588,0.819935


In [13]:
if not all(trainData.columns == predictData.columns):
    raise Exception("Column do not match!")

In [14]:
#8 training 
labels = list(labelsData.columns)
problem_types = ['regression'] * len(labels)  
eval_metrics = ['mean_squared_error'] * len(labels)
time_limit = 60 * 60 * (24) * 3
save_path = 'agModels-predictDrugAUC_hyperparametertunning'  

In [15]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, eval_metrics=eval_metrics, path=save_path)
multi_predictor.fit(trainDataSet, time_limit=time_limit)
#multi_predictor = MultilabelPredictor.load(save_path)

In [17]:
#result = multi_predictor.predict(predictData)
#result.to_csv("result_hyperparametertunning_dasatinb.csv")

In [20]:
predictor = multi_predictor.get_predictor("dasatinib")

In [23]:
predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            WeightedEnsemble_L2  -4.407888      31.269618   163.301895                0.000530           0.704748            2       True        359
1     NeuralNetTorch_BAG_L1/T108  -4.620973       3.185626    25.828971                3.185626          25.828971            1       True        337
2       NeuralNetTorch_BAG_L1/T8  -4.654633       2.809808    24.454505                2.809808          24.454505            1       True        237
3      NeuralNetTorch_BAG_L1/T63  -4.674536       0.467185    16.443708                0.467185          16.443708            1       True        292
4      NeuralNetTorch_BAG_L1/T40  -4.683050       2.406467    25.741998                2.406467          25.741998            1       True        269
5     NeuralNetTorch_BAG_L1/T126  -4.7

*** End of fit() summary ***


{'model_types': {'KNeighborsUnif_BAG_L1/T1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1/T1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1/T1': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T2': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T3': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T4': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T5': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T6': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T7': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T8': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T9': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T10': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T11': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T12': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T13': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T14': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T15': 'StackerEnsembleModel_LGB',
  'LightGBMXT_BAG_L1/T16': 'StackerEnsembleModel_LGB'

In [ ]:
predictor.info()